### Introduction

Reference: https://ostwalprasad.github.io/machine-learning/Bayesian-Linear-Regression-using-PyMC3.html

> In statistics, Bayesian linear regression is an approach to linear regression in which the statistical analysis is undertaken within the context of Bayesian inference. When the regression model has errors that have a normal distribution, and if a particular form of prior distribution is assumed, explicit results are available for the posterior probability distributions of the model's parameters.

**Linear Regression**


In simple linear regression we get point estimates by,
$$ y = \alpha\ + \beta\ *x $$
 Equation says, there's a linear relationship between variable $x$ and $y$. Slope is controlled by $ \beta\ $ and intercept tells about value of $y$ when $x=0$ . Methods like Ordinary Least Squares, optimize the parameters to minimize the error between observed $y$ and predicted $y$. These methods only return single best value for parameters.
  
** Bayesian Approach: **
 
 The same problem can be stated under probablistic framework. We can obtain best values of $ \alpha\ $  and $ \beta\ $ along with their uncertainity estimations. Probablistically linear regression can be explained as : 
 
$$ y \sim N(\mu=\alpha+\beta x, \sigma=\varepsilon) $$

$y$ is observed as a Gaussian distribution with mean $ \mu\ $ and standard deviation $ \sigma\ $. Unlike OLS regression, here it is normally distibuted. Since we do not know the values of $ \alpha\ $  , $ \beta\ $ and $ \epsilon\ $, we have to set prior distributions for them. 

$$ \begin{array}{l}{\alpha \sim N\left(\mu_{\alpha}, \sigma_{\alpha}\right)} \\ {\beta \sim N\left(\mu_{\beta}, \sigma_{\beta}\right)} \\ {\varepsilon \sim U\left(0, h_{s}\right)}\end{array}$$




In this post, I'm going to demonstrate very simple linear regression problem with both OLS and bayesian approach. We will use [PyMC3 package](https://docs.pymc.io/). PyMC3 is a Python package for Bayesian statistical modeling and probabilistic machine learning.

###Import basic modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


### Generate linear artificial data

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = 2 * x - 5 + 2*rng.randn(50)
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

plt.scatter(x, y)

In [ ]:
# If you want to install pymc3 in Anaconda, check this: https://anaconda.org/conda-forge/pymc3
# conda install -c conda-forge pymc3
! pip install pymc3

### Create PyMC3 model

In [5]:
import pymc3 as pm
print('Running on the PyMC3 v{}'.format(pm.__version__))
basic_model =  pm.Model()

Running on the PyMC3 v3.11.5


### Define model parameters

Context is created for defining model parameters using `with` statement. Using context makes it easy to assign parameters to model.

Distributions for  $ \alpha\ $  , $ \beta\ $ and $ \epsilon\ $ are defined. $ \mu\ $ is a deterministic variable which calculated using line equation.

`Ylikelihood` is a likelihood parameter which is defined ny Normal distribution with $ \mu\ $ and $ \sigma\ $. Observed values are also passed along with distribution.

In [6]:
with basic_model as bm:

    #Priors
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10)
    sigma = pm.HalfNormal('sigma', sd=1)

    # Deterministics
    mu = alpha + beta*x
    
    # Likelihood 
    Ylikelihood = pm.Normal('Ylikelihood', mu=mu, sd=sigma, observed=y)

### Sampling from posterior

As model is defined completely, now we can sample from posterior. PyMC3 automatically chooses appropriate model depending on the type of data. In our case of continuous data, NUTS is used.

In [ ]:
 trace = pm.sample(draws=2000,model=bm)

### Trace summary and plots
Traceplots plots samples histograms and values.

In [ ]:
pm.traceplot(trace)

In [ ]:
print(pm.summary(trace).round(2))

### Checking autocorrelation

Bar plot of the autocorrelation function for a trace can be plotted using [pymc3.plots.autocorrplot](https://docs.pymc.io/api/plots.html).

Autocorrelation dictates the amount of time you have to wait for convergence. If autocorrelation is high, you will have to use a longer burn-in. Low autocorrelation means good exploration.


In [ ]:
pm.autocorrplot(trace)

### Comparing parameters with Simple Linear Regression (OLS)

Parameters can be cross checked using Simple Linear Regression.

In [ ]:

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
ypred =  lm.fit(x,y).predict(x)
plt.scatter(x,y)
plt.plot(x,ypred)
legend_title = 'Simple Linear Regression\n {} + {}*x'.format(round(lm.intercept_[0],2),round(lm.coef_[0][0],2))
legend = plt.legend(loc='upper left', frameon=False, title=legend_title)
plt.title("Simple Linear Regression")
plt.show()
